In [1]:
import sys,os
from collections import Counter
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq, reverse_complement
import difflib
import fuzzysearch
import pandas as pd
import pickle

In [2]:
#Globals
wd = os.getcwd()
user_profile = os.environ ['USERPROFILE']
retrondb = '%s/retrondb_for_RTDNA_census.csv' % (wd) #for getting ncRNAs

polyA_adapt = 'AAAAAAAAAAGATCGGAAGAGCACACGTCTGAACTCC'
polyC_adapt = 'CCCCCCAGATCGGAAGAGCACACGTCTGAACTCC'

sub = 6 #which subset to analyze
pickled_globals = True #either False if it's the first subset or True if it's not

In [3]:
#load retrondb from backup csv for ncRNAs
rdb = pd.read_csv(retrondb)
#load key of conditions and files
samples = pd.read_excel("MiSeq_data_key_RTDNAseq.xls")[["MiSeq file","DBR1 status","nodes","subset"]]

In [4]:
#Global variables
if pickled_globals == False:
    node_list_with_repeats = []
    for i in samples.index.values:
        node_list_with_repeats.extend(list(map(int,samples.loc[i]['nodes'].split(','))))
    node_list = list(set(node_list_with_repeats))

    ncRNA_list = []
    ncRNA_dict = {}
    ncRNA_count = {}
    ncRNA_count_rel_to_Eco1 = {}
    for node in node_list:
        ncRNA_list.append(rdb.loc[rdb.node == str(node),'ncRNA'].values[0])
        ncRNA_dict[rdb.loc[rdb.node == str(node),'ncRNA'].values[0]] = node
        ncRNA_count[node] = 0
    coverage_dict_pA = {} #main key is node, under each node is a dictionary of base postition with counts of that base
    for sequence in ncRNA_dict:
        coverage_dict_pA[ncRNA_dict[sequence]] = {} #this will make a new dictionary for each node
        for i in range(0,len(sequence)):
            coverage_dict_pA[ncRNA_dict[sequence]][i] = 0

    coverage_dict_pC = {} #main key is node, under each node is a dictionary of base postition with counts of that base
    for sequence in ncRNA_dict:
        coverage_dict_pC[ncRNA_dict[sequence]] = {} #this will make a new dictionary for each node
        for i in range(0,len(sequence)):
            coverage_dict_pC[ncRNA_dict[sequence]][i] = 0

    coverage_dict_pA_minusDBR1 = {} #main key is node, under each node is a dictionary of base postition with counts of that base
    for sequence in ncRNA_dict:
        coverage_dict_pA_minusDBR1[ncRNA_dict[sequence]] = {} #this will make a new dictionary for each node
        for i in range(0,len(sequence)):
            coverage_dict_pA_minusDBR1[ncRNA_dict[sequence]][i] = 0

    coverage_dict_pC_minusDBR1 = {} #main key is node, under each node is a dictionary of base postition with counts of that base
    for sequence in ncRNA_dict:
        coverage_dict_pC_minusDBR1[ncRNA_dict[sequence]] = {} #this will make a new dictionary for each node
        for i in range(0,len(sequence)):
            coverage_dict_pC_minusDBR1[ncRNA_dict[sequence]][i] = 0

    fidelity_dict = {} #main key is node, under each node is a dictionary of total matched bases and incorrect bases (errors), includes both pA and pC
    for sequence in ncRNA_dict:
        fidelity_dict[ncRNA_dict[sequence]] = {'total_bases':0,'error_bases':0} #this will make a new dictionary for each node

else:
    node_list_with_repeats = []
    for i in samples.index.values:
        node_list_with_repeats.extend(list(map(int,samples.loc[i]['nodes'].split(','))))
    node_list = list(set(node_list_with_repeats))

    ncRNA_list = []
    ncRNA_dict = {}
    for node in node_list:
        ncRNA_list.append(rdb.loc[rdb.node == str(node),'ncRNA'].values[0])
        ncRNA_dict[rdb.loc[rdb.node == str(node),'ncRNA'].values[0]] = node
    
    with open('pickled_%s.pickle' %(str(sub-1)),"rb") as f: #note that this is based on the order they were written into the pickle
        ncRNA_count = pickle.load(f)
        ncRNA_count_rel_to_Eco1 = pickle.load(f)
        coverage_dict_pA = pickle.load(f)
        coverage_dict_pC = pickle.load(f)
        coverage_dict_pA_minusDBR1 = pickle.load(f)
        coverage_dict_pC_minusDBR1 = pickle.load(f)
        fidelity_dict = pickle.load(f)

In [5]:
#adapter trimming and binning
#input fastq
#output total seqs, no_initial_T, no_R_adapter, just_ligated_adapters, RTDNA_shorter_than_10, pA_trimmed, pC_trimmed
def adapter_trim(fastq):
    seqs_in_sample = 0
    just_ligated_adapters = []
    RTDNA_shorter_than_10 = []  
    no_initial_T = []
    no_R_adapter = []
    pA_trimmed = []
    pC_trimmed = []
    for seq_record in SeqIO.parse(fastq, "fastq"):
        seqs_in_sample += 1
        if str(seq_record.seq)[0] != 'T':
            no_initial_T.append(seq_record)
        else:
            T_trimmed = str(seq_record.seq)[1:]
            polyA_split = T_trimmed.split(polyA_adapt)
            if len(polyA_split) == 2:
                if polyA_split[0] == '':
                    just_ligated_adapters.append(seq_record)
                elif len(polyA_split[0]) < 11:
                    RTDNA_shorter_than_10.append(seq_record)
                else: 
                    pA_trimmed.append(polyA_split[0])
            else:
                polyC_split = T_trimmed.split(polyC_adapt)
                if len(polyC_split) == 2:
                    if polyC_split[0] == '':
                        just_ligated_adapters.append(seq_record)
                    elif len(polyC_split[0]) < 11:
                        RTDNA_shorter_than_10.append(seq_record)
                    else: 
                        pC_trimmed.append(polyC_split[0])
                else: no_R_adapter.append(seq_record) 
    return (seqs_in_sample, no_initial_T, no_R_adapter, just_ligated_adapters, RTDNA_shorter_than_10, pA_trimmed, pC_trimmed)

In [6]:
#bin by retron, just pA
def match_and_analyze(DBR1_status, A_or_C, nodes, counter):
    internal_ncRNA_list = []
    ncRNA_count_pA = {}
    for node in nodes:
        internal_ncRNA_list.append(rdb.loc[rdb.node == str(node),'ncRNA'].values[0])
        ncRNA_count_pA[node] = 0
    unmatched_count = 0
    one_matched_count = 0
    double_matched_count = 0
    odd_bucket = 0
    binned_to_node_but_no_fuzzy_match = []
    global coverage_dict_pA
    global coverage_dict_pC
    global coverage_dict_pA_minusDBR1
    global coverage_dict_pC_minusDBR1
    global fidelity_dict
    global ncRNA_count
    if DBR1_status == 'DBR1+':
        if A_or_C == 'A':
            coverage_dict = coverage_dict_pA
        else: 
            coverage_dict = coverage_dict_pC
    else:
        if A_or_C == 'A':
            coverage_dict = coverage_dict_pA_minusDBR1
        else: 
            coverage_dict = coverage_dict_pC_minusDBR1      
    for key in counter:
        R_seq = reverse_complement(key) #reverse complement to put it in the same dirction as the ncRNA
        ncRNA_match = difflib.get_close_matches(R_seq,internal_ncRNA_list)
        if len(ncRNA_match) == 1:  #this is the case we care about where it found one unique match in the ncRNA list
            node = ncRNA_dict[ncRNA_match[0]]
            ncRNA_count[node] += counter[key]
            one_matched_count += counter[key]
            match_loc = fuzzysearch.find_near_matches(R_seq, ncRNA_match[0], max_l_dist=int(len(R_seq)*0.1)) #allowing 10% mismatch, too high?
            if len(match_loc): #some sequences are found with difflib, but don't match closely enough to work with fuzzysearch
                for i in range(match_loc[0].start,match_loc[0].end):
                    coverage_dict[node][i] += counter[key]
                if DBR1_status == 'DBR1+':  #only use the DBR1 data for fidelity
                    fidelity_dict[node]['total_bases'] += (match_loc[0].end-match_loc[0].start)*counter[key]
                    fidelity_dict[node]['error_bases'] += match_loc[0].dist*counter[key]
                    if A_or_C == 'A':
                        ncRNA_count_pA[node] += counter[key] #only use pA, DBR1 data for counts relative to Eco1
            else:
                binned_to_node_but_no_fuzzy_match.append(key)
        elif len(ncRNA_match) > 1:
            double_matched_count += counter[key]
        elif len(ncRNA_match) < 1:
            unmatched_count += counter[key]
        else: odd_bucket += 1
    if DBR1_status == 'DBR1+' and A_or_C == 'A': 
        return (unmatched_count,one_matched_count,double_matched_count,odd_bucket,binned_to_node_but_no_fuzzy_match,ncRNA_count_pA)
    else:    
        return (unmatched_count,one_matched_count,double_matched_count,odd_bucket,binned_to_node_but_no_fuzzy_match)


In [ ]:
sub_df = samples.loc[samples['subset'] == sub]

for i in sub_df.index.values:
    fastq_reads = '%s/%s.fastq' % (wd, samples.loc[i]['MiSeq file'])
    seqs_in_sample, no_initial_T, no_R_adapter, just_ligated_adapters, RTDNA_shorter_than_10, pA_trimmed, pC_trimmed = adapter_trim(fastq_reads)
    print (samples.loc[i]['MiSeq file'])
    print ('Total Seqs: %s' % (seqs_in_sample))
    print ('pA_trimmed: %s' % len(pA_trimmed))
    print ('pC_trimmed: %s' % len(pC_trimmed))
    print ('just_ligated_adapters: %s' % len(just_ligated_adapters))
    print ('RTDNA_shorter_than_10: %s' % len(RTDNA_shorter_than_10))
    print ('no_initial_T: %s' % len(no_initial_T))
    print ('no_R_adapter: %s' % len(no_R_adapter) + '\n')
    pA_counter = Counter(pA_trimmed)
    pC_counter = Counter(pC_trimmed)
    #analyze pA
    if samples.loc[i][1] == 'DBR1+':
        unmatched_count,one_matched_count,double_matched_count,odd_bucket,binned_to_node_but_no_fuzzy_match,ncRNA_count_pA = match_and_analyze(samples.loc[i][1], 'A', list(map(int,samples.loc[i]['nodes'].split(','))), pA_counter)
        #deal with relative counts 
        for node in ncRNA_count_pA:
            current = float(ncRNA_count_pA[node])/ncRNA_count_pA[1550]
            try:
                prev = ncRNA_count_rel_to_Eco1[node]
                ncRNA_count_rel_to_Eco1[node] = (prev+current)/2
            except KeyError:
                ncRNA_count_rel_to_Eco1[node] = current
    else:
        unmatched_count,one_matched_count,double_matched_count,odd_bucket,binned_to_node_but_no_fuzzy_match = match_and_analyze(samples.loc[i][1], 'A', list(map(int,samples.loc[i]['nodes'].split(','))), pA_counter)   
    print ('pA reads')
    print ('unmatched count: %s' % (unmatched_count))
    print ('one match count: %s' % (one_matched_count))
    print ('double matched count: %s' % (double_matched_count))
    print ('binned to node but no fuzzy match: %s' % (len(binned_to_node_but_no_fuzzy_match)))
    print ('odd leftovers: %s' % (odd_bucket) + '\n')
    #analyze pC
    unmatched_count,one_matched_count,double_matched_count,odd_bucket,binned_to_node_but_no_fuzzy_match = match_and_analyze(samples.loc[i][1], 'C', list(map(int,samples.loc[i]['nodes'].split(','))), pC_counter)
    print ('pC reads')
    print ('unmatched count: %s' % (unmatched_count))
    print ('one match count: %s' % (one_matched_count))
    print ('double matched count: %s' % (double_matched_count))
    print ('binned to node but no fuzzy match: %s' % (len(binned_to_node_but_no_fuzzy_match)))
    print ('odd leftovers: %s' % (odd_bucket) + '\n')

with open('pickled_%s.pickle' %(str(sub)),"wb") as f:
    pickle.dump(ncRNA_count,f)
    pickle.dump(ncRNA_count_rel_to_Eco1,f)
    pickle.dump(coverage_dict_pA,f)
    pickle.dump(coverage_dict_pC,f)
    pickle.dump(coverage_dict_pA_minusDBR1,f)
    pickle.dump(coverage_dict_pC_minusDBR1,f)
    pickle.dump(fidelity_dict,f)

In [8]:
#make dataframe for output, only run this after the final subset
coverage_dict_pA_minusDBR1
output_data = {'node':[],
               'RTDNAseq_count_pA':[],
               'RTDNAseq_count_pC':[],
               'RTDNAseq_count_pA_minusDBR1':[],
               'RTDNAseq_count_pC_minusDBR1':[],
               'RTDNAseq_errors_per_base':[],
               'RTDNAseq_count_rel_to_Eco1':[]}              
for node in node_list:
    output_data['node'].append(node)
    cov_list_pA = [str(coverage_dict_pA[node][key]) for key in sorted(coverage_dict_pA[node].keys())]
    cov_list_pC = [str(coverage_dict_pC[node][key]) for key in sorted(coverage_dict_pC[node].keys())]
    cov_list_pA_minusDBR1 = [str(coverage_dict_pA_minusDBR1[node][key]) for key in sorted(coverage_dict_pA_minusDBR1[node].keys())]
    cov_list_pC_minusDBR1 = [str(coverage_dict_pC_minusDBR1[node][key]) for key in sorted(coverage_dict_pC_minusDBR1[node].keys())]    
    output_data['RTDNAseq_count_pA'].append(";".join(cov_list_pA))
    output_data['RTDNAseq_count_pC'].append(";".join(cov_list_pC))
    output_data['RTDNAseq_count_pA_minusDBR1'].append(";".join(cov_list_pA_minusDBR1))
    output_data['RTDNAseq_count_pC_minusDBR1'].append(";".join(cov_list_pC_minusDBR1))
    output_data['RTDNAseq_count_rel_to_Eco1'].append(ncRNA_count_rel_to_Eco1[node])
    if fidelity_dict[node]['total_bases'] > 0:  
        output_data['RTDNAseq_errors_per_base'].append(fidelity_dict[node]['error_bases']/fidelity_dict[node]['total_bases'])
    else: output_data['RTDNAseq_errors_per_base'].append('div/0')
RTDNA_seq_analysis = pd.DataFrame(output_data)
RTDNA_seq_analysis.to_csv("RTDNA_seq_analysis.csv",index=False)